# Structural analysis of discontinuous concepts

The purpose of this analysis notebook is to make an in-depth analysis of structures. This makes the basis for a large portion of the corresponding section in the final thesis paper.

## Coordinated concepts

Coordinations are the only thing marked in GENIA while CRAFT also contains other types. Therefore, GENIA will be used primarily in this part. The variable `corpus_name` can be set to either `genia` or `craft` to run with.

In [138]:
corpus_name = 'genia'

First, let's load the corpus, then extract concepts and their constituent structures.

In [139]:
import os
os.chdir('..')  # get to the root directory of the project

from datautils import dataio, annotations as anno

# load the corpora
corpus = dataio.load_corpus(corpus_name)

# not all docs in genia get Constituent annotations; if so, leave them out
if corpus_name.lower() == 'genia':
    corpus = [doc for doc in corpus if doc.get_annotations(anno.Constituent)]

Loading GENIA corpus ... NOTE: 13 files cannot get Constituent annotations!


100%|██████████| 1599/1599 [00:19<00:00, 83.40it/s] 


In [140]:
concept_consts = [(concept, doc.get_annotations_at(concept.span, anno.Constituent)[0])
                  for doc in corpus
                  for concept in doc.get_annotations(anno.DiscontinuousConcept)
                  if len(concept.get_tokens()) < 10]

# for craft, keep only the ones with a conjunction in them
temp_list = []
for concept, const in concept_consts:
    present_pos = {t.mapped_pos() for t in const.get_tokens()}
    if 'c' in present_pos:
        temp_list.append((concept, const))
concept_consts = temp_list

Each constituent can spit out a treebank structure. Let's do that and put it all in dataframes.

In [141]:
import re

def skipped_structure(const, allowed_tokens):
    """Returns  a treebank structure of a Constituent with a # symbol in front of
    all tokens that are in allowed_tokens"""
    # note: craft constituents often have labels like NP-SBJ; keep only the first part
    return '(' + const.label.split('-')[0] + ' ' + ' '.join(
        skipped_structure(c, allowed_tokens) if isinstance(c, anno.Constituent)
        else c.mapped_pos() if c not in allowed_tokens
        else '#' + c.mapped_pos()
        for c in const.constituents
    ) + ')'

def unlabeled_structure(const, allowed_tokens):
    """Returns a simplified treebank structure of a Constituent with no constituent labels
    and with a # symbol in front of all tokens that are in allowed_tokens"""
    return_string = '(' + ' '.join(
        unlabeled_structure(c, allowed_tokens) if isinstance(c, anno.Constituent)
        else c.mapped_pos() if c not in allowed_tokens
        else '#' + c.mapped_pos()
        for c in const.constituents
        if not contains_empty_token(c)
    ) + ')'
    # remove all cases of single token constituents, e.g. (n)
    return re.sub('\(+(#?.)\)+', r'\1', return_string)

def contains_empty_token(const):
    if not isinstance(const, anno.Constituent):
        return False
    for c in const.constituents:
        if isinstance(c, anno.Token) and c.mapped_pos() == 'Ø':
            return True
    return False

In [142]:
import pandas as pd
from collections import defaultdict

full_structures = defaultdict(list)
skip_structures = defaultdict(list)
collapsed_full_structures = defaultdict(list)
collapsed_skip_structures = defaultdict(list)

for concept, const in concept_consts:
    struct = const.structure()  
    full_structures[struct].append(const)
    
    skip_struct = skipped_structure(const, concept.get_concept_tokens())
    skip_structures[skip_struct].append(const)
    
    collapsed_full = unlabeled_structure(const, {})
    collapsed_full_structures[collapsed_full].append(const)
    
    collapsed_skip = unlabeled_structure(const, concept.get_concept_tokens())
    collapsed_skip_structures[collapsed_skip].append(const)

In [143]:
n_dcs = len(concept_consts)

dfs = {}

for name, structure_dict in [('full', full_structures),
                             ('skip', skip_structures),
                             ('col_full', collapsed_full_structures),
                             ('col_skip', collapsed_skip_structures)
                            ]:
    
    data_dict = {'struct': [], 'pos-seq': [], 'count': [], '%': [], 'example': []}
    
    for struct, sample in structure_dict.items():
        data_dict['struct'].append(struct)
        data_dict['pos-seq'].append(''.join(c for c in struct if c.islower() or c in ','))
        data_dict['count'].append(len(sample))
        data_dict['%'].append(round(len(sample) / n_dcs * 100, 2))
        data_dict['example'].append(sample[0].get_covered_text())
        
    dfs[name] = pd.DataFrame(data_dict)

In [144]:
dfs['col_skip'].sort_values('count', ascending=False).head(50)

,struct,pos-seq,count,%,example
13,(#n c n #n),ncnn,83,5.84,T and NK cells
9,(#a c a #n),acan,69,4.86,Positive and negative regulation
0,(t #n c n #n),tncnn,44,3.10,the FY*A and FY*B alleles
6,((#n c n) #n),ncnn,33,2.32,Egr-1 and c-fos mRNA
50,((#a c a) #n),acan,21,1.48,control and infected cells
15,(#n c n (#n #n)),ncnnn,20,1.41,HLA-DMA and HLA-DMB gene expression
36,(#n c (f #a) #n),ncfan,14,0.99,TNF-alpha- or PMA-induced stimulation
29,(#a c a (#n #n)),acann,14,0.99,bacterial or mycobacterial response elements
107,((#a c a) #n #n),acann,12,0.84,endocrine and exocrine precursor cells
215,((#n c (f #a)) #n),ncfan,12,0.84,glucocorticoid- and Fas-mediated cytotoxicity


In [145]:
# for latex table
import re
csv_string = dfs['full'].sort_values('count', ascending=False).head(15).to_csv(sep='&')
csv_string = re.sub(r'\n\d+&', r'  \\\\ \\addlinespace\n', csv_string)  # line breaks and row numbers
csv_string = re.sub('^&', r'', csv_string)  # first &
csv_string = re.sub('%', r'\\%', csv_string)  # \% for latex
csv_string = re.sub('Ø', r'\\O', csv_string)  # \O for latex
csv_string = re.sub('#([A-Za-z\-]+)', r'\\textbf{\1}', csv_string)  # bold typeface used words 
csv_string = re.sub('&', r'\t&\t', csv_string)  # more space on delimiters

In [146]:
print(csv_string)

struct	&	pos-seq	&	count	&	\%	&	example  \\ \addlinespace
(NP (NP n (NP \O)) c (NP n (NP \O)) (NP n))	&	ncnn	&	70	&	4.93	&	T and NK cells  \\ \addlinespace
(NP (NP (ADJP a) (NP \O)) c (NP (ADJP a) (NP \O)) (NP n))	&	acan	&	61	&	4.29	&	Positive and negative regulation  \\ \addlinespace
(NP t (NP n (NP \O)) c (NP n (NP \O)) (NP n))	&	tncnn	&	36	&	2.53	&	the FY*A and FY*B alleles  \\ \addlinespace
(NP (NP (NP n) c (NP n)) n)	&	ncnn	&	32	&	2.25	&	Egr-1 and c-fos mRNA  \\ \addlinespace
(NP (ADJP (ADJP a) c (ADJP a)) n)	&	acan	&	20	&	1.41	&	monocytic and erythroid differentiation  \\ \addlinespace
(NP (NP n (NP \O)) c (NP n (NP \O)) (NP n n))	&	ncnnn	&	20	&	1.41	&	HLA-DMA and HLA-DMB gene expression  \\ \addlinespace
(NP (NP n (ADJP \O) (NP \O)) c (NP f a (NP \O)) (NP n))	&	ncfan	&	13	&	0.91	&	TNF-alpha- or PMA-induced stimulation  \\ \addlinespace
(NP t (NP (NP n) c (NP n)) n)	&	tncnn	&	12	&	0.84	&	the AP-1 and Elf-1 sites  \\ \addlinespace
(NP (ADJP (ADJP a) c (ADJP a)) n n)	&	acann	&	12	&

In [147]:
collapsed_skip_structures['((#a c a) #n)']

[Constituent('control and infected cells'(535, 561))\NP,
 Constituent('monocytic and erythroid differentiation'(54, 93))\NP,
 Constituent('bronchial and alveolar cells'(786, 814))\NP,
 Constituent('steroid-sensitive and steroid-resistant asthma'(69, 115))\NP,
 Constituent('severe and symptomatic VKC'(309, 335))\NP,
 Constituent('humoral or cell-mediated immunity'(476, 509))\NP,
 Constituent('immune and inflammatory responses'(303, 336))\NP,
 Constituent('immune and inflammatory responses'(124, 157))\NP,
 Constituent('haematopoietic and solid tumours'(387, 419))\NP,
 Constituent('immune and inflammatory responses'(353, 386))\NP,
 Constituent('neonatal and adult cells'(1127, 1151))\NP,
 Constituent('basal and Tax-mediated transcription'(851, 887))\NP,
 Constituent('oxidant and antioxidant conditions'(1924, 1958))\NP,
 Constituent('Murine and human TCF-1'(703, 725))\NP,
 Constituent('homodimeric or heterodimeric forms'(1351, 1385))\NP,
 Constituent('cellular and viral genes'(205, 229))\NP

We can capture more examples under fewer labels if we only use POS-tag sequences.

In [148]:
CROSS_COUNTING = ['concept', 'super', 'skip']  # order: concept, super, skip
SUPER_SEQ_COLLAPSERS = {
    # re.compile('n*(n,?)+cnn+'),
    # re.compile('a*(a,?)+caa*n+')
    re.compile('[an]c[an]n'),  # base case: shared head
    re.compile('[an]ncn'),  # base case: shared modifier
    re.compile('([an],)+[an],?c[an]n'),  # exp case: enumeration of modifiers + shared head
    re.compile('[an]+(n,)+n,?cn'),  # exp case: enumeration of heads with shared modifier
    re.compile('[an]c[an][an]+n'),  # exp case: multi-word head
    re.compile('[an]+[an]ncn'),  # exp case: shared pre-modifiers
}
CONCEPT_SEQ_COLLAPSERS = {
    re.compile('(a|n)+n')
}
SAMPLE_CATEGORIES = {'concept', 'super', 'skip', 'cross-count'}

def make_sequence(tokens, collapser=None):
    sequence = [t.mapped_pos() if not t.pos == '#' else '#'
                for t in tokens]
    sequence_str = ''.join(sequence)
    if not collapser:
        return sequence_str

    for pos_pattern in collapser:
        if pos_pattern.fullmatch(sequence_str):
            return pos_pattern.pattern
    return sequence_str  # sequence did not match a pattern

In [149]:
disc_concepts = [dc for doc in corpus
                 for dc in doc.get_annotations(anno.DiscontinuousConcept)
                 if 'c' in {t.mapped_pos() for t in dc.get_tokens()}
                 and len(dc.get_tokens()) < 10]

samples_after_category = {}
for ct in SAMPLE_CATEGORIES:
    samples_after_category[ct] = defaultdict(list)

for dc in disc_concepts:
    concept_tokens = dc.get_concept_tokens()
    all_tokens = dc.get_tokens()
    skip_tokens = [t if t in concept_tokens  # actual token
                   else anno.Token(None, (-1, -1), '#')  # skipped token
                   for t in all_tokens]

    cross_count_type = []
    cross_count_example = []

    concept_sequence = make_sequence(concept_tokens, CONCEPT_SEQ_COLLAPSERS)
    concept_text = dc.get_concept()
    samples_after_category['concept'][concept_sequence].append(dc)
    if 'concept' in CROSS_COUNTING:
        cross_count_type.append(concept_sequence)
        cross_count_example.append(dc)

    super_sequence = make_sequence(all_tokens, SUPER_SEQ_COLLAPSERS)
    super_text = dc.get_covered_text()
    samples_after_category['super'][super_sequence].append(dc)
    if 'super' in CROSS_COUNTING:
        cross_count_type.append(super_sequence)
        cross_count_example.append(dc)

    skip_sequence = make_sequence(skip_tokens)
    samples_after_category['skip'][skip_sequence].append(dc)
    if 'skip' in CROSS_COUNTING:
        cross_count_type.append(skip_sequence)
        cross_count_example.append(dc)

    samples_after_category['cross-count'][tuple(cross_count_type)].append(
        tuple(cross_count_example)
    )

In [150]:
data_dict = {'type': [], 'count': [], '%': [], 'example': []}
for type_, items in samples_after_category['super'].items():
    data_dict['type'].append(type_)
    data_dict['count'].append(len(items))
    data_dict['%'].append(round(len(items) / len(disc_concepts) * 100, 2))
    data_dict['example'].append(items[0].get_covered_text())
    
df = pd.DataFrame(data_dict)

In [151]:
df.sort_values('count', ascending=False).head(20)

,type,count,%,example
0,[an]c[an]n,407,29.56,FY*A and FY*B alleles
3,[an]c[an][an]+n,230,16.70,Egr-1 and c-fos mRNA expression
15,ncfan,45,3.27,TNF-alpha- or PMA-induced stimulation
39,"([an],)+[an],?c[an]n",37,2.69,"STAT6, STAT3 and STAT5 proteins"
12,nncnn,35,2.54,PTF gamma and delta subunits
73,ancnn,19,1.38,human T and B lymphocytes
7,[an]ncn,16,1.16,DP alpha and -beta
14,ncfann,16,1.16,N- and C- terminal SH2 domains
21,ndcd,15,1.09,positions -585 and -650
13,[an]+[an]ncn,15,1.09,B cell proliferation and differentiation


In [152]:
for item in samples_after_category['super']['[an]+[an]ncn']:
    print(*(t.get_covered_text() for t in item.get_tokens()))

B cell proliferation and differentiation
human glial cells and lymphocytes
PKC isoenzymes alpha and beta
T cell activation and growth
human GM-CSF promoter and enhancer
T lymphocyte differentiation and activation
T lymphocyte activation and mitogenesis
T cell activation and differentiation
viral gene expression and replication
viral transcription initiation and elongation
human chromosomes 11p15 and 11p13
HTLV-I basal transcription and expression
human B cell proliferation and differentiation
IL-4 receptor subunits alpha and gamma
T cell receptor alpha and -beta


In [154]:
# for exp case: multi-word head: for multiple tokens after the conjunction,
# do the tokens go with the head or the second modifier?
df = dfs['col_full']
subset = df[df['pos-seq'].str.contains('^[an]+[an]ncn$')]
subset.head(25)

,struct,pos-seq,count,%,example
26,(n n n c n),nnncn,2,0.14,B cell proliferation and differentiation
88,(a (a n) c n),aancn,1,0.07,human glial cells and lymphocytes
93,(n n (n c n)),nnncn,3,0.21,PKC isoenzymes alpha and beta
272,(a n n c n),anncn,2,0.14,viral gene expression and replication
328,(a n (n c n)),anncn,1,0.07,human chromosomes 11p15 and 11p13
409,(n ((a n) c n)),nancn,1,0.07,HTLV-I basal transcription and expression
411,(a n n (n c n)),annncn,1,0.07,human B cell proliferation and differentiation
525,(n n n n c n),nnnncn,1,0.07,T cell receptor alpha and -beta
